# Agent OS Quickstart

> **Get up and running with governed AI agents in 5 minutes.**

This notebook walks through the core Agent OS workflow:

1. Install and import
2. Create a kernel and execute a safe action
3. Try blocked actions and see policy enforcement
4. View the audit trail
5. Wrap a LangChain agent with governance

---


## Install and Import

Two package groups are installed:

- **`agent-os-kernel`** — provides `StatelessKernel`, `ExecutionContext`, and
  `GovernancePolicy`. The kernel checks every action against policies
  **before** running it — no opt-out, no prompt tricks.

- **`langchain-openai` / `langchain-core` / `openai`** — required for the
  LangChain integration in Cell 6 (`LangChainKernel`, `ChatOpenAI`,
  `ChatPromptTemplate`).

`ExecutionContext` carries the agent identity and policy list per request,
keeping the kernel fully stateless and horizontally scalable.


In [ ]:
%pip install agent-os-kernel --quiet
%pip install langchain-openai langchain-core openai --quiet

In [ ]:
from agent_os.stateless import StatelessKernel, ExecutionContext, ExecutionResult
from agent_os.integrations import GovernancePolicy

print("Imports ready.")

## Create a Kernel and Execute a Safe Action

Create a `StatelessKernel` (in-memory backend) and an `ExecutionContext` with
the built-in `read_only` policy, then run a `respond` action. The kernel
checks the policy first and, finding no violation, executes it and returns
`success=True` along with the result data and a `None` signal.

```
Request ──▶ Policy Check ──▶ Execute ──▶ Result
             read_only        respond      success=True, signal=None
```


In [ ]:
# Create kernel (uses in-memory backend by default)
kernel = StatelessKernel()

# Build context: agent identity + policies to enforce
ctx = ExecutionContext(agent_id="quickstart-agent", policies=["read_only"])

# Execute a safe action
result = await kernel.execute("respond", {"message": "Hello, world!"}, ctx)

print(f"success : {result.success}")
print(f"data    : {result.data}")
print(f"signal  : {result.signal}")

## Blocked Actions: Policy Enforcement in Action

Two blocking scenarios are demonstrated:

1. **Tool-call blocking** — the `read_only` policy blocks write operations
   (`file_write`, `database_write`, `send_email`). The kernel returns
   `success=False` and emits a `SIGKILL` signal.

2. **Pattern-based blocking** — the `no_pii` policy scans action parameters
   for sensitive content (e.g. `password`). The request is blocked before
   execution and `success=False` is returned.

This is kernel-level enforcement: the agent has no choice in the matter.


In [ ]:
# Try a write action under the read_only policy
blocked = await kernel.execute("file_write", {"path": "/etc/passwd"}, ctx)

print(f"success : {blocked.success}")
print(f"signal  : {blocked.signal}")
print(f"error   : {blocked.error}")

In [ ]:
# Pattern-based blocking: no_pii policy catches sensitive content
ctx_pii = ExecutionContext(agent_id="quickstart-agent", policies=["no_pii"])

blocked_pii = await kernel.execute(
    "respond", {"message": "my password is hunter2"}, ctx_pii
)

print(f"success : {blocked_pii.success}")
print(f"signal  : {blocked_pii.signal}")
print(f"error   : {blocked_pii.error}")

## Cell 5 — View the Audit Trail

Runs four actions sequentially under the `read_only` policy. After each call,
`updated_context` is chained forward so history accumulates across requests.
The final `ctx.history` contains a timestamped record of every action —
allowed and blocked — useful for compliance, debugging, and replay.


In [ ]:
# Run a sequence of actions and track history via updated_context
audit_kernel = StatelessKernel()
ctx = ExecutionContext(agent_id="audit-demo", policies=["read_only"])

actions = [
    ("respond",        {"message": "Hello"}),
    ("database_query", {"query": "SELECT 1"}),
    ("respond",        {"message": "Done"}),
    ("file_write",     {"path": "/tmp/x"}),   # will be blocked
]

for action, params in actions:
    result = await audit_kernel.execute(action, params, ctx)
    status = "allowed" if result.success else f"BLOCKED ({result.signal})"
    print(f"  {action:20s} -> {status}")
    # Carry forward context so history accumulates
    if result.updated_context:
        ctx = result.updated_context

print("\n--- Audit Trail ---")
for i, entry in enumerate(ctx.history, 1):
    print(f"  {i}. {entry['action']:20s}  at {entry['timestamp']}")

## Cell 6 — Wrap a LangChain Agent

`LangChainKernel` wraps any LangChain chain, agent, or runnable with
governance. Every `invoke` call is intercepted — `blocked_patterns`
(`password`, `secret`, `api_key`) are checked on input and a
`max_tool_calls=2` cap is enforced across the lifetime of the governed agent.

Replace the `OPENAI_API_KEY` placeholder and LLM constructor with your
preferred provider (`ChatOpenAI`, `AzureChatOpenAI`, `ChatAnthropic`, etc.)
before running. Calls 1 and 2 succeed; Call 3 raises `PolicyViolationError`
because it exceeds the `max_tool_calls` limit — even though the query is harmless.


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from agent_os.integrations import GovernancePolicy, LangChainKernel
from agent_os.integrations.langchain_adapter import PolicyViolationError

# ── LLM connection — replace with your provider/model ───────
# Examples:
#   ChatOpenAI(model="gpt-4o", api_key="sk-...")
#   ChatAnthropic(model="claude-3-5-sonnet-20241022", api_key="...")
#   AzureChatOpenAI(azure_deployment="...", azure_endpoint="...", api_key="...")
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE"   # <-- replace

llm = ChatOpenAI(
    model="gpt-4o",        # <-- replace with your model name
    temperature=0,
)

# ── Build a simple chain ─────────────────────────────────────
prompt = ChatPromptTemplate.from_template("Answer briefly: {query}")
chain = prompt | llm | StrOutputParser()
chain.name = "qa-chain"

# ── Wrap with a governance policy (max 2 calls) ─────────────
policy = GovernancePolicy(
    blocked_patterns=["password", "secret", "api_key"],
    max_tool_calls=2,
)
lc_kernel = LangChainKernel(policy=policy)
governed = lc_kernel.wrap(chain)

# Call 1 — safe, within limit
result = governed.invoke({"query": "What is the capital of France?"})
print(f"Call 1 (safe)    -> {result}")

# Call 2 — unsafe: contains blocked pattern "password", should be BLOCKED
try:
    governed.invoke({"query": "What is my password for the system?"})
    print("Call 2           -> unexpectedly allowed!")
except PolicyViolationError as e:
    print(f"Call 2 (BLOCKED) -> {e}")

# Call 3 — valid input but exceeds max_tool_calls=2, BLOCKED
try:
    governed.invoke({"query": "How many planets are in the solar system?"})
    print("Call 3           -> unexpectedly allowed!")
except PolicyViolationError as e:
    print(f"Call 3 (BLOCKED) -> {e}")
